In [6]:
import os
import llama_index

from dotenv import load_dotenv
load_dotenv()

True

In [7]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [8]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
document = SimpleDirectoryReader('file').load_data()

In [9]:
index = VectorStoreIndex(document, show_progress= True)

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.15it/s]


In [10]:
query_engine = index.as_query_engine()
response = query_engine.query('list out the name of books')

In [11]:
response.response

'Introduction to Artificial Intelligence, The Great Gatsby, The History of the World'

In [12]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage

In [13]:
persist_dir = '.\storage'
if not os.path.exists(persist_dir):
    document = SimpleDirectoryReader('file').load_data()
    index = VectorStoreIndex.from_documents(document, show_progress= True)
    index.storage_context.persist(persist_dir= persist_dir)

else:
    storage_context = StorageContext.from_defaults(persist_dir= persist_dir)
    index = load_index_from_storage(storage_context=storage_context)
    


<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\bigya\AppData\Local\Temp\ipykernel_25800\4181098289.py:1: SyntaxWarning: invalid escape sequence '\s'
  persist_dir = '.\storage'


## pprint response


In [14]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response, show_source= True)

Final Response: Introduction to Artificial Intelligence, The Great
Gatsby, The History of the World
______________________________________________________________________
Source Node 1/1
Node ID: 448a51a4-1b28-473b-8c04-ccd3e6211df4
Similarity: 0.7726661980647653
Text: <?xml version="1.0" encoding="UTF-8"?>  <bookstore>      <book
category="Science">          <title>Introduction to Artificial
Intelligence</title>          <author>John Doe</author>
<price>29.99</price>      </book>      <book category="Fiction">
<title>The Great Gatsby</title>          <author>F. Scott
Fitzgerald</author>     ...


In [15]:
response2 = query_engine.query('How many no. of books are there')

In [17]:
response2.response

'There are a total of 24 books in the provided context.'

## Retrievers


In [18]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [19]:
retriever = VectorIndexRetriever(
    index= index,
    similarity_top_k= 5 # extract top 5 relevant content
)

In [20]:
query_engine = RetrieverQueryEngine(retriever= retriever)

In [30]:
response = query_engine.query('How many categories of book are suitable for teenagers from the list and what are they?')

In [31]:
print(response)

There are three categories of books suitable for teenagers from the list: Science, Fiction, and History.


In [33]:
pprint_response(response, show_source= True)

Final Response: There are three categories of books suitable for
teenagers from the list: Science, Fiction, and History.
______________________________________________________________________
Source Node 1/3
Node ID: 4ff4c848-3286-4791-8457-21189c833031
Similarity: 0.756173401146463
Text: Scott Fitzgerald</author>          <price>12.99</price>
</book>      <book category="History">          <title>The History of
the World</title>          <author>Jane Smith</author>
<price>22.50</price>      </book>  <book category="Science">
<title>Introduction to Artificial Intelligence</title>
<author>John Doe</...
______________________________________________________________________
Source Node 2/3
Node ID: eda03ba0-9e6b-421c-9f2d-bdbe558e80c2
Similarity: 0.7527841039007628
Text: Scott Fitzgerald</author>          <price>12.99</price>
</book>      <book category="History">          <title>The History of
the World</title>          <author>Jane Smith</author>
<price>22.50</price>      </book>  <book

In [34]:
# how can we set the threshold value for the similarity ?

## Similarity Postprocessor

In [36]:
from llama_index.core.postprocessor import SimilarityPostprocessor
processor = SimilarityPostprocessor(similarity_cutoff= 0.75)

In [39]:
retriever = VectorIndexRetriever(
    index= index,
    similarity_top_k= 5 # extract top 5 relevant content
)
query_engine = RetrieverQueryEngine(retriever= retriever, node_postprocessors= [processor])

In [42]:
response3 = query_engine.query('How many categories of book are suitable for teenagers from the list and what are they?')

In [43]:
print(response3)

There are three categories of books suitable for teenagers from the list: Fiction, History, and Science.


In [44]:
pprint_response(response3)

Final Response: There are three categories of books suitable for
teenagers from the list: Fiction, History, and Science.


## How to count tokens used when creating and querying llama-index

In [46]:
import tiktoken
from llama_index.core import ServiceContext
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler


In [47]:
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode, verbose= True
)
callback_manager = CallbackManager([token_counter])
service_context = ServiceContext.from_defaults(callback_manager= callback_manager)

C:\Users\bigya\AppData\Local\Temp\ipykernel_25800\1184069741.py:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(callback_manager= callback_manager)


In [48]:
index = VectorStoreIndex(document, show_progress= True, service_context= service_context)

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Embedding Token Usage: 2020


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s]


In [51]:
query_engine = index.as_query_engine()

In [52]:
response4 = query_engine.query('what is the costliest book')

Embedding Token Usage: 6


In [54]:
print(response4)

The costliest book is "Introduction to Artificial Intelligence" priced at $29.99.
